In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark
findspark.init()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/My Drive/LDS9_K272_ONLINE_DoThiPhuong/LDS9_K272_DoThiPhuong_Cuoi_ky/'

/content/gdrive/My Drive/LDS9_K272_ONLINE_DoThiPhuong/LDS9_K272_DoThiPhuong_Cuoi_ky


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF, Tokenizer
from pyspark.ml.feature import NGram

In [ ]:
MAX_MEMORY = '10g'
spark = SparkSession.builder\
                    .appName('cau_2')\
                    .config('spark.executor.memory', MAX_MEMORY)\
                    .config('spark.driver.memory', MAX_MEMORY)\
                    .getOrCreate()

In [ ]:
fake = spark.read.csv('Data/fake-and-real-news-dataset/Fake.csv', inferSchema=True, header = True)

In [ ]:
real = spark.read.csv('Data/fake-and-real-news-dataset/True.csv', inferSchema=True, header = True)

In [ ]:
fake.show(5)

+--------------------+--------------------+-------+-----------------+
|               title|                text|subject|             date|
+--------------------+--------------------+-------+-----------------+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017|
+--------------------+--------------------+-------+-----------------+
only showing top 5 rows



In [ ]:
real.show(5)

+--------------------+--------------------+------------+------------------+
|               title|                text|     subject|              date|
+--------------------+--------------------+------------+------------------+
|As U.S. budget fi...|WASHINGTON (Reute...|politicsNews|December 31, 2017 |
|U.S. military to ...|WASHINGTON (Reute...|politicsNews|December 29, 2017 |
|Senior U.S. Repub...|WASHINGTON (Reute...|politicsNews|December 31, 2017 |
|FBI Russia probe ...|WASHINGTON (Reute...|politicsNews|December 30, 2017 |
|Trump wants Posta...|SEATTLE/WASHINGTO...|politicsNews|December 29, 2017 |
+--------------------+--------------------+------------+------------------+
only showing top 5 rows



In [ ]:
fake.columns

['title', 'text', 'subject', 'date']

In [ ]:
real.columns

['title', 'text', 'subject', 'date']

In [ ]:
from pyspark.sql.functions import lit

In [ ]:
fake = fake.withColumn('class', lit('fake'))
fake.show(5)

+--------------------+--------------------+-------+-----------------+-----+
|               title|                text|subject|             date|class|
+--------------------+--------------------+-------+-----------------+-----+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017| fake|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017| fake|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017| fake|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017| fake|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017| fake|
+--------------------+--------------------+-------+-----------------+-----+
only showing top 5 rows



In [ ]:
real = real.withColumn('class', lit('real'))
real.show(5)

+--------------------+--------------------+------------+------------------+-----+
|               title|                text|     subject|              date|class|
+--------------------+--------------------+------------+------------------+-----+
|As U.S. budget fi...|WASHINGTON (Reute...|politicsNews|December 31, 2017 | real|
|U.S. military to ...|WASHINGTON (Reute...|politicsNews|December 29, 2017 | real|
|Senior U.S. Repub...|WASHINGTON (Reute...|politicsNews|December 31, 2017 | real|
|FBI Russia probe ...|WASHINGTON (Reute...|politicsNews|December 30, 2017 | real|
|Trump wants Posta...|SEATTLE/WASHINGTO...|politicsNews|December 29, 2017 | real|
+--------------------+--------------------+------------+------------------+-----+
only showing top 5 rows



In [ ]:
fake.count()

23489

In [ ]:
fake.printSchema()

root
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- date: string (nullable = true)
 |-- class: string (nullable = false)



In [ ]:
real.count()

21417

In [ ]:
real.printSchema()

root
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- date: string (nullable = true)
 |-- class: string (nullable = false)



In [ ]:
data = fake.union(real)
data.show(5)

+--------------------+--------------------+-------+-----------------+-----+
|               title|                text|subject|             date|class|
+--------------------+--------------------+-------+-----------------+-----+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017| fake|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017| fake|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017| fake|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017| fake|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017| fake|
+--------------------+--------------------+-------+-----------------+-----+
only showing top 5 rows



In [ ]:
data.columns

['title', 'text', 'subject', 'date', 'class']

In [ ]:
data.count()

44906

In [ ]:
data.show(5)

+--------------------+--------------------+-------+-----------------+-----+
|               title|                text|subject|             date|class|
+--------------------+--------------------+-------+-----------------+-----+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017| fake|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017| fake|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017| fake|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017| fake|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017| fake|
+--------------------+--------------------+-------+-----------------+-----+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import length

In [ ]:
data = data.withColumn('length', length(data['text']))

In [ ]:
data.show(5)

+--------------------+--------------------+-------+-----------------+-----+------+
|               title|                text|subject|             date|class|length|
+--------------------+--------------------+-------+-----------------+-----+------+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017| fake|  2893|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017| fake|  1898|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017| fake|  3597|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017| fake|  2774|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017| fake|  2346|
+--------------------+--------------------+-------+-----------------+-----+------+
only showing top 5 rows



In [ ]:
data.groupby('class').mean().show()

+-----+------------------+
|class|       avg(length)|
+-----+------------------+
| real| 2372.866507914274|
| fake|2476.0307482645544|
+-----+------------------+



In [ ]:
data.groupby('class').count().show()

+-----+-----+
|class|count|
+-----+-----+
| real|21417|
| fake|23489|
+-----+-----+



In [ ]:
data = data.select(['text', 'class', 'length'])
data.show(5)

+--------------------+-----+------+
|                text|class|length|
+--------------------+-----+------+
|Donald Trump just...| fake|  2893|
|House Intelligenc...| fake|  1898|
|On Friday, it was...| fake|  3597|
|On Christmas day,...| fake|  2774|
|Pope Francis used...| fake|  2346|
+--------------------+-----+------+
only showing top 5 rows



In [ ]:
data.describe().show()

+-------+--------------------+-----+------------------+
|summary|                text|class|            length|
+-------+--------------------+-----+------------------+
|  count|               44898|44906|             44898|
|   mean|                null| null|2426.8199028909976|
| stddev|                null| null| 2156.224782504735|
|    min|                    | fake|                 1|
|    max|youngers these da...| real|             51794|
+-------+--------------------+-----+------------------+



In [ ]:
data = data.filter(data.length >= 20)
data.count()

44228

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [ ]:
class_to_num = StringIndexer(inputCol= 'class', outputCol='label')
tokenizer = Tokenizer(inputCol= 'text', outputCol='token_text')
stopremove = StopWordsRemover(inputCol= 'token_text', outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol= 'stop_tokens', outputCol='c_vec', vocabSize= 400, minDF=100)
idf = IDF(inputCol= 'c_vec', outputCol='tf_idf')

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [ ]:
clean_up = VectorAssembler(inputCols= ['tf_idf', 'length'], outputCol='features')

In [ ]:
from pyspark.ml.classification import NaiveBayes

In [ ]:
nb= NaiveBayes()

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
data_prep_pipe = Pipeline(stages= [class_to_num,
                                   tokenizer,
                                   stopremove,
                                   count_vec,
                                   idf,
                                   clean_up])

In [ ]:
cleaner = data_prep_pipe.fit(data)
clean_data = cleaner.transform(data)

In [ ]:
clean_data = clean_data.select(['label', 'features'])

In [ ]:
clean_data.show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(401,[0,1,4,5,6,8...|
|  0.0|(401,[0,1,2,4,9,1...|
|  0.0|(401,[0,1,5,6,10,...|
|  0.0|(401,[0,1,4,5,8,1...|
|  0.0|(401,[0,1,2,6,10,...|
|  0.0|(401,[0,5,29,37,3...|
|  0.0|(401,[0,1,10,21,3...|
|  0.0|(401,[0,1,4,5,9,1...|
|  0.0|(401,[0,1,3,5,6,1...|
|  0.0|(401,[0,1,4,5,10,...|
+-----+--------------------+
only showing top 10 rows



In [ ]:
(training, testing) = clean_data.randomSplit([0.7, 0.3])

In [ ]:
predictor = nb.fit(training)

In [ ]:
data.printSchema()

root
 |-- text: string (nullable = true)
 |-- class: string (nullable = false)
 |-- length: integer (nullable = true)



In [ ]:
test_results = predictor.transform(testing)

In [ ]:
test_results.show(10)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(401,[0,1,2,3,4,5...|[-2419.3404312761...|[1.0,1.1843951192...|       0.0|
|  0.0|(401,[0,1,2,3,4,5...|[-8312.5609353186...|[0.99633787142055...|       0.0|
|  0.0|(401,[0,1,2,3,4,5...|[-10050.063326995...|[1.0,6.2062208081...|       0.0|
|  0.0|(401,[0,1,2,3,4,5...|[-2082.8422521264...|[1.0,2.9701294899...|       0.0|
|  0.0|(401,[0,1,2,3,4,5...|[-7907.7534230406...|[1.0,2.4080074963...|       0.0|
|  0.0|(401,[0,1,2,3,4,5...|[-1679.6123869613...|[1.0,5.2046959392...|       0.0|
|  0.0|(401,[0,1,2,3,4,5...|[-1287.2656348693...|[0.99999999994688...|       0.0|
|  0.0|(401,[0,1,2,3,4,5...|[-2476.9045736253...|[1.0,3.3444806364...|       0.0|
|  0.0|(401,[0,1,2,3,4,5...|[-8183.5242178325...|[1.0,3.1681321272...|       0.0|
|  0.0|(401,[0,1

In [ ]:
test_results.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0| 6382|
|  0.0|       1.0|  456|
|  1.0|       0.0|  129|
|  0.0|       0.0| 6379|
+-----+----------+-----+



In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print('accuracy:', acc)

accuracy: 0.9561664002713456


In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol= 'prediction')

In [ ]:
AUC = my_eval.evaluate(test_results)
AUC

0.9567359699757442

In [ ]:
TN = test_results.filter('prediction = 0 AND label = prediction').count()
TP = test_results.filter('prediction = 1 AND label = prediction').count()
FN = test_results.filter('prediction = 0 AND label != prediction').count()
FP = test_results.filter('prediction = 1 AND label != prediction').count()

In [ ]:
precision = TP / (TP + FP)
recall = TP / (TP + FN)
print('precision = {:.2f}\nrecall = {:.2f}'.format(precision, recall))

precision = 0.93
recall = 0.98


In [ ]:
## model dự đoán tốt